# Create vcf file with all possible peptide elongations resulting from a SNV in a STOP codon
1. Get the nucleotide sequences of all canonical peptides
    - Downloaded Coding Sequences (cds) from ensembl (https://ftp.ensembl.org/pub/release-112/fasta/homo_sapiens/).

    Function *parse_cds_to_dict()*
    - 1.0 Check if gene_symbol already processed previously. If yes, skip to next.
    - 1.1 Find first ATG in cds, if not move to next and print to std.out
    - 1.2 Check for downstream STOP codon, if not print std.out
    - 1.3 If ATG and STOP found, store gene_symbol, ENSG, ENST and description in df
    - 1.4 Store gene_symbol and CDS in seq object
    - 1.5 Mitochondrial genes will be dropped as they follow diffferent rules of translationa nd transcription and often have no UTRs.

    After this step, we obtain 19 360 CDS sequences of unique genes that have a "valid sequence"

2. Get cDNA with ENST-ID of CDS
    - 2.1 Create a cdna_dict from cDNA ensmbl file with ENST_ID:cDNA_seq (function *build_cdna_dict()*)
    Function *extract_enstID()* to get transcript ID of CDS entry
    Function *create_enst_dict()* creates dictionary with gene:ENST_ID
    - 2.2 Function *add_cdna_to_cds()* to match CDS with cDNA using the transcript ID (ENST)
    
    For 892 of the 19 360 CDS, we found no matching ENST-ID in the cDNA.

3. Mutating the STOP codon of the 18 467 CDS
    - 3.1 Function *trim_cdna_to_overlap()* will keep only the 3'UTR of the cDNA (seq after end of CDS).
    - 3.2 Function *find_ds_STOP_in3UTR()* checks for downstream STOP codons in the 3'UTR. If none found, the entry will be discarded.
    Out of 18468, we removed 300 genes due to missing dsSTOP (1.62%).
    - 3.3 Function *generate_mutated_sequences_with_utr()* introduces all possible non-silent mutations in the original STOP codon of the CDS. 
    The possibile mutated codons are stored in nonsilentSNV_dict.
    - 3.4 Function *translate_mutated_cds_to_pep()* uses Bio.Seq object to translate the elongtaed CDS to peptide

4. Write fasta files
    - 4.1 Function *update_header()* will create a unique ID for each mutated seq in the format >[gene_symbol]_n 
    - 4.2 Function *write_fasta_files()* creates 2 output files:
        - peptide seq in amino acids
        - mutated and elongated CDS in nucleotides

## Output:
The final two fasta files contain the exact same sequneces of the mutated sequences
in amino acid (pep) and nucleotide (CDS) format.
Mutated sequence: [8 last codons of CDS] + [mutated STOP codon] + [3'UTR to downstream STOP codon]

In [38]:
import sys
import pandas as pd
from collections import Counter
wdir = '/users/genomics/lilian/STOPcodons_in_humangenes/STOPloss_canonicalPeptides/'
outpath = wdir+'data/'

old_stdout = sys.stdout
log_file = open(wdir+'scripts/01_get_seq_elongation_v2.0.log', 'w')
sys.stdout = log_file

## Human Genome files. ensembl version 112

In [39]:
# Ref files
GRCh38_cds_fasta = "/data/genomics/lilian/Ref_genomes/GRCh38/Homo_sapiens.GRCh38.cds.all.fa"
GRCh38_cDNA_fasta = "/data/genomics/lilian/Ref_genomes/GRCh38/Homo_sapiens.GRCh38.cdna.all.fa"

### STEP 1: Get the CDS for all unique genes

One gene symbol can have several cds. We will keep the first one that starts with ATG and ends with one of the three common STOP codons.

- Check if gene_symbol already processed previously. If yes, skip to next.
- Check if transcript_biotype:protein_coding. If no, skip to next.
- Keep only sequences:
    1. ... starting with ATG
    2. ... ending with a STOP codon
    3. ... having STOP in-frame with START codon (seq_len = multiple of 3)
- Check if termiates in STOP codon, if not print std.out
- Store gene with full header and CDS in final dictionary


In [40]:
def parse_cds_to_dict(cds_filepath):
    """
    Extracting CDS for all unique genes into dictionary
    Only considering transcripts with "transcript_biotype:protein_coding" in their header
    Will take the first cds transcript matching 
    """
    unique_cds_dict = {}
    current_gene = None
    current_sequence = []
    handled_genes = [] # list to store all genes with transcript biotype protein_coding

    with open(cds_filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if "transcript_biotype:protein_coding" not in line:
                    current_gene = None  # Skip this entry
                    continue

                # If current gene and sequence, store to dictionary
                if current_gene is not None and current_gene not in unique_cds_dict:
                    cds = ''.join(current_sequence)

                    # add seq to dictionary 1.) IF starts with ATG and 2.) IF contains in-frame STOP 
                    start_index = cds.find('ATG')
                    # If no ATG found, return False
                    if start_index == -1:
                        print(f"Skipping entry for {current_gene}: sequence does not start with 'ATG'.")

                    # if ATG found, check for in-frame stop codons (TAG, TGA, TAA)
                    no_stop_found = True
                    for i in range(start_index, len(cds), 3):
                        codon = cds[i:i+3]
                        if codon in ['TAG', 'TGA', 'TAA']:
                            no_stop_found = False
                            #cds_start_stop = full_sequence[start_index:i+3]
                            unique_cds_dict[current_gene] = {'cds_info': current_header, 
                                                             'cds_sequence': cds, 
                                                             'ATG_pos': start_index,
                                                             'STOP_pos': i}
                    if no_stop_found:
                        print(f"Skipping entry for {current_gene}: sequence misses expected STOP codon.")

                current_header = line
                # get gene_symbol from header
                header = line.split()
                for part in header:
                    if part.startswith("gene_symbol:"):
                        current_gene = part.split(":")[1].split('.')[0] 
                        handled_genes.append(current_gene)

                # if gene_symbol is already in dictionary, skip lines
                if current_gene in unique_cds_dict:
                    current_gene = None  

                # Reset the current sequence list for the new gene
                current_sequence = []
            else:
                 # Only append sequence if we have a valid current_gene
                if current_gene is not None:
                    current_sequence.append(line)

        # After finishing the loop, store the last gene and sequence
        if current_gene is not None and current_gene not in unique_cds_dict:
            cds = ''.join(current_sequence)

            # add seq to dictionary 1.) IF starts with ATG and 2.) IF contains in-frame STOP 
            start_index = cds.find('ATG')
            # If no ATG found, return False
            if start_index == -1:
                print(f"Skipping entry for {current_gene}: sequence does not start with 'ATG'.")

            # if ATG found, check for in-frame stop codons (TAG, TGA, TAA)
            no_stop_found = True
            for i in range(start_index, len(cds), 3):
                codon = cds[i:i+3]
                if codon in ['TAG', 'TGA', 'TAA']:
                    no_stop_found = False
                    #cds_start_stop = full_sequence[start_index:i+3]
                    unique_cds_dict[current_gene] = {'cds_info': current_header, 
                                                        'cds_sequence': cds, 
                                                        'ATG_pos': start_index,
                                                        'STOP_pos': i}
            if no_stop_found:
                print(f"Skipping entry for {current_gene}: sequence misses expected STOP codon.")


    return unique_cds_dict

In [41]:
print("Load CDS into dict ...")
cds_dict = parse_cds_to_dict(GRCh38_cds_fasta)
print(f"\nNumber of unique genes with cds in dictionary:{len(cds_dict.keys())}")

### STEP 2: Get cDNA, containing 3'UTR after STOP
- from header_info in cds_dict, take the transcript_ID
- grep transcript_ID in cDNA file

In [42]:
def extract_enstID(header_info):
    """
    Get ENST-ID from header line in format 
    '>ENST00000289823.1 cdna chromosome:NCBI35:8:21922367:21927699:1 gene:ENSG00000158815.1 gene_biotype:protein_coding transcript_biotype:protein_coding'
    """
    enst_id = header_info.split()[0][1:]  # Remove '>' and extract ENST ID
    
    # Check if the extracted ID starts with 'ENST'
    if enst_id.startswith('ENST'):
        return enst_id
    else:
        print(f"Invalid ENST ID in header: {header_info}")
        return None

In [43]:
def create_enst_dict(cds_dict):
    """
    Create a dictionary mapping gene_symbols to their corresponding ENST IDs.
    """
    enst_dict = {}
    for gene, info in cds_dict.items():
        enst_id = extract_enstID(info['cds_info'])
        if enst_id:
            enst_dict[gene] = enst_id  # Map gene name to ENST ID
    return enst_dict

In [44]:
def build_cdna_dict(cdna_filepath):
    """
    Build a dictionary with ENST IDs and cDNA seq
    """
    cdna_dict = {}
    current_enst_id = None
    current_sequence = []

    with open(cdna_filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                # If we have an ENST ID and a current sequence, store it in the dictionary
                if current_enst_id is not None:
                    cdna_dict[current_enst_id] = ''.join(current_sequence)

                # Extract the new ENST ID from the current header
                current_enst_id = extract_enstID(line)
                current_sequence = []  # Reset sequence for the new capture
            elif current_enst_id is not None:
                current_sequence.append(line)

        # Store the last entry if present
        if current_enst_id is not None:
            cdna_dict[current_enst_id] = ''.join(current_sequence)

    return cdna_dict

In [45]:
def add_cdna_to_cds(cds_infodict, enst_dict, cdna_dict):
    """
    Add cDNA sequences from cdna_dict to the cds_dict as a new item to each gene's entry in cds_dict.
    """
    genes_without_cDNA = []

    for gene_symbol, info in cds_infodict.items():
        # Get the ENST ID for the current gene symbol
        enst_id = enst_dict.get(gene_symbol)
        
        if enst_id:
            # Check if the ENST ID exists in the cDNA dictionary
            cdna_seq = cdna_dict.get(enst_id)
            if cdna_seq:
                # Add the cDNA sequence to the cds_dict entry
                info['cdna_seq'] = cdna_seq
            else:
                print(f"cDNA sequence for ENST ID {enst_id} not found.")
                # remove cases where no cDNA was found for ENST-ID
                genes_without_cDNA.append(gene_symbol)
        else:
            print(f"ENST ID for gene {gene_symbol} not found in enst_dict.")

    # Remove the marked genes from cds_dict
    for gene_symbol in genes_without_cDNA:
        del cds_infodict[gene_symbol]

    return cds_infodict

In [46]:
print("\n\nBuild dictionarry with gene_symbol:ENST_ID")
enst_dict = create_enst_dict(cds_dict)

print("\n\nBuild cDNA dictionary with ENST_ID:cDNA")
cdna_dict = build_cdna_dict(GRCh38_cDNA_fasta)

print("\n\nFor ENST_ID with CDS, grep the cDNA seq")
gene_cdna_dict = {}
n_ENST_notfound=0
for gene, enst_id in enst_dict.items():
    if enst_id in cdna_dict:
        gene_cdna_dict[gene] = cdna_dict[enst_id]
    else:
        print(f"ENST ID {enst_id} for gene {gene} not found in the cDNA file.")
        n_ENST_notfound+=1

pct_noENSTID = n_ENST_notfound / (len(cds_dict.keys()) /100)
pct_noENSTID = str(round(pct_noENSTID,2))

print(f"\n\nWe are working with {len(cdna_dict.keys())} cDNA sequences.")
print(f"From {len(cds_dict.keys())} genes, {n_ENST_notfound} don't have a CDS with matched cDNA ENST-IDs ({pct_noENSTID}%).")

In [47]:
print("\nWe will drop all mitochondrial genes as theire translaction is more complex and follows different rules than nuclear genes (e.g. often no UTRs)")
# Dropping all genes marked with "MT-"
cds_dict_noMT = {k: v for k, v in cds_dict.items() if not k.startswith('MT-')}
print(f"By dropping MT genes, we went from",len(cds_dict.keys()),"to",len(cds_dict_noMT.keys()),"number of cds', dropping",len(cds_dict.keys())-len(cds_dict_noMT.keys()),"genes")

In [48]:
len(enst_dict.keys()) # 18290
len(cdna_dict.keys()) # 206723 cDNAs
len(cds_dict_noMT.keys()) # 18277 CDS matched with cDNA and ENST, removing mitochondrials

18277

In [49]:
print("\n\nAdd the cDNA as a new item to the cds_dict")
complete_dict = add_cdna_to_cds(cds_dict_noMT, enst_dict, cdna_dict)
print(f"\n\nWe continue with {len(complete_dict.keys())} genes.") #17369 genes

### STEP 3: Mutate STOP codon
**Step 3.1: Function *trim_cdna_to_overlap()* will keep only the 3'UTR of the cDNA (seq after end of CDS).**
For all CDS with valid sequences and matched cDNA:
- Drop the 5'UTR from the cDNAseq (keeping overlap + downstream cDNA)

In [50]:
def append_3UTR(sequences_dict, similarity_threshold=0.98):
    """
    Find the overlapping sequence from the Stop of the cds to the end of the cdna_seq (3'UTR),
    tolerating small differences between sequences.
    Returns dictionary with the same genes as keys but adds a new item values are the overlapped sequences
     """
    original_stop_codons = []

    def find_stop_codon(sequence):
        """Find the first stop codon (TAA, TAG, or TGA) in the cds.
        Return the position of the first stop codon found. """
        stop_codons = ['TAA', 'TAG', 'TGA']
        
        # Find the first ATG
        start_pos = sequence.find('ATG')
        if start_pos == -1:
            return -1,-1, ''
        # Read sequence in triplets starting from ATG
        pos = start_pos
        while pos + 3 <= len(sequence):
            codon = sequence[pos:pos + 3]
            if codon in stop_codons:
                return start_pos, pos, codon
            pos += 3  # Move to next codon
            
        # if no stop codon found
        return -1, -1, ''
        

    def calculate_similarity(seq1, seq2):
        """Calculate the ratio of matching nucleotides between two sequences."""
        if len(seq1) == 0:
            return 0
        if len(seq1) != len(seq2):
            return 0
        matches = sum(n1 == n2 for n1, n2 in zip(seq1, seq2))
        return matches / len(seq1)
    
    def find_best_start_position(cds_start, cdna_seq):
        """Find the best matching position using a sliding window."""
        best_score = 0
        best_pos = -1
        
        # Use the start of CDS sequence as a reference window
        if len(cds_start) < 20 :
            window_size = len(cds_start)
        else:
            window_size = 20
        cds_window = cds_start[:window_size]


        # Slide the window through cdna_seq
        for i in range(len(cdna_seq) - window_size + 1):
            cdna_window = cdna_seq[i:i + window_size]
            score = calculate_similarity(cds_window, cdna_window)
            
            if score > best_score:
                best_score = score
                best_pos = i
                
            # Early exit if we find a very good match
            if score > 0.95:
                break
                
        return best_pos, best_score


    trimmed_cDNA_dict = {}

    for gene_id, info in sequences_dict.items():
        # check if both seq exist
        if 'cds_sequence' not in info or 'cdna_seq' not in info:
            raise KeyError(f"Missing required sequence keys for {gene_id}")
   
        # Handle case-insensitive matching and leading N's
        cds_seq = info['cds_sequence'].strip('Nn').upper()
        cdna_seq = info['cdna_seq'].strip('Nn').upper()
        
        # Find where cds_seq starts in cdna_seq
        cds_start_pos, similarity = find_best_start_position(cds_seq, cdna_seq)
        
        if cds_start_pos == -1 or similarity < similarity_threshold:
            print(   f"No sufficient match found for {gene_id}. "
                                f"Best similarity: {similarity:.2f}" )

        # Find the stop codon in the overlapping cds region
        aligned_seq = cdna_seq[cds_start_pos:] 
        start_pos, stop_pos, stop_codon = find_stop_codon(aligned_seq)

        original_stop_codons.append(stop_codon)

        # cut cds from start pos to end pos
        cds_cut = aligned_seq[start_pos : stop_pos+3]

        # Extract sequence from stop codon to end (including stop codon)
        trimmed_cdna_seq = aligned_seq[stop_pos + 3:]

        trimmed_cDNA_dict[gene_id] = {
            'header' : info.get('cds_info'),  
            'CDS' : cds_cut, 
            'CDS_uncut' : cds_seq,
            'cDNA' : cdna_seq,
            '3UTR' : trimmed_cdna_seq  
        }
    return trimmed_cDNA_dict,original_stop_codons

print("\n\nAppend 3'UTR from cDNA to cds.")
complete_dict_3UTR,original_stop_codons = append_3UTR(complete_dict)
# 18468, no cases removed due to cDNA-CDS unmatched pairs

In [51]:
print(f"We continue working with",len(complete_dict_3UTR.keys()),"transcripts.") # 17356

In [52]:
complete_dict_3UTR['PTMA']

{'header': '>ENST00000409321.5 cds chromosome:GRCh38:2:231707684:231713108:1 gene:ENSG00000187514.17 gene_biotype:protein_coding transcript_biotype:protein_coding gene_symbol:PTMA description:prothymosin alpha [Source:HGNC Symbol;Acc:HGNC:9623]',
 'CDS': 'ATGAATCCGTGTGTGCTTGGGACGGTGCCCGCTTGTCGACTGACTTGGCCGACGGACGCCGGACCTGATGACGCAGCCGTAGACACCAGCTCCGAAATCACCACCAAGGACTTAAAGGAGAAGAAGGAAGTTGTGGAAGAGGCAGAAAATGGAAGAGACGCCCCTGCTAACGGGAATGCTAATGAGGAAAATGGGGAGCAGGAGGCTGACAATGAGGTAGACGAAGAAGAGGAAGAAGGTGGGGAGGAAGAGGAGGAGGAAGAAGAAGGTGATGGTGAGGAAGAGGATGGAGATGAAGATGAGGAAGCTGAGTCAGCTACGGGCAAGCGGGCAGCTGAAGATGATGAGGATGACGATGTCGATACCAAGAAGCAGAAGACCGACGAGGATGACTAG',
 'CDS_uncut': 'ATGAATCCGTGTGTGCTTGGGACGGTGCCCGCTTGTCGACTGACTTGGCCGACGGACGCCGGACCTGATGACGCAGCCGTAGACACCAGCTCCGAAATCACCACCAAGGACTTAAAGGAGAAGAAGGAAGTTGTGGAAGAGGCAGAAAATGGAAGAGACGCCCCTGCTAACGGGAATGCTAATGAGGAAAATGGGGAGCAGGAGGCTGACAATGAGGTAGACGAAGAAGAGGAAGAAGGTGGGGAGGAAGAGGAGGAGGAAGAAGAAGGTGATGGTGAGGAAGAGGATGGAGATGAAGATGAGGAAGCTGAGTCAGCTACGGGCAAGCGG

### STEP 4: Getting seq of peptide extensions
4.1 Create all possible mutations of cds


4.2 Find next ds STOP
4.2 translate to next ds STOP, if no dsSTop discard

### STEP 4: Getting seq of peptide extensions
4.1 Get downstream STOP codon in 3'UTR
Since we can be sure that all cds seq in our dictionary are in the correct reading frame,
we will just take the last 9 amino acids (8*3+3=27 nucleotides) of each seq
 (we controled)
- passing 3codon window to find downstream STOP
- if STOP found, cut 3'UTR to STOP
- if no STOP in 3'UTR, delete entry

In [53]:
def find_ds_STOP_in3UTR(trimmed_dict):
    """
    Scan 3'UTR in frames of 3 nucleotides to look for a ds STOP.
    If a ds STOP is found, trim 3'UTR to that position.
    If no in-frame STOP is found, entry is dropped from dictionary.
    
    Returns a new dictionary with the 3'UTR trimmed.
    """
    trimmed_UTR_dict = {}
    no_dsSTOP_found = []
    counter = 0
    stop_codons = ['TAA', 'TAG', 'TGA']
    ds_STOPcodon = []

    for gene_symbol, info in trimmed_dict.items():
        utr_3_seq = info.get('3UTR')

        # Slide through the 3' UTR in frames of 3 to find a downstream STOP codon
        stop_found = False
        for i in range(0, len(utr_3_seq), 3):
            codon = utr_3_seq[i:i+3]
            if codon in stop_codons:
                stop_found = True
                # store new stop for statistics
                ds_STOPcodon.append(codon)
                # Trim the 3' UTR up to and including this STOP codon
                trimmed_UTR_seq = utr_3_seq[:i + 3]  # Keep up to STOP codon
                # Create a new entry with the trimmed cdna_seq
                trimmed_UTR_dict[gene_symbol] = {
                    'header': info.get('header'),  
                    'CDS': info.get('CDS'),  
                    'cDNA': info.get('cDNA'),
                    '3UTR': info.get('3UTR'),
                    'UTR_trimmed': trimmed_UTR_seq  
                }
                break

        # If no downstream STOP codon is found, print a warning and remove gene from dict
        if not stop_found:
            no_dsSTOP_found.append(gene_symbol)
            counter += 1
            print(f"No downstream in-frame STOP codon found for gene {gene_symbol}. Entry will be removed.")

    genes_before = len(trimmed_dict.keys())
    pctt_genes_noSTOP = counter / ( genes_before /100) 
    pctt_genes_noSTOP = str(round(pctt_genes_noSTOP,2))
    print(f"Out of {len(trimmed_dict.keys())}, we removed {counter} genes due to missing dsSTOP ({pctt_genes_noSTOP}%).")

    return trimmed_UTR_dict,ds_STOPcodon

print("\n\nDrop 3'UTR after the first ds STROP codon.")
trimmed_UTR_dict, new_ds_STOPcodons = find_ds_STOP_in3UTR(complete_dict_3UTR)

Compare if the distributin of STOP codons changes between the cds and the ds STOPs in the 3'UTR

In [54]:
from scipy.stats import chi2_contingency
import numpy as np
import matplotlib.pyplot as plt

stop_codons = ['TGA', 'TAG', 'TAA']

# Convert Counters to lists of observed frequencies
original_counter = Counter(original_stop_codons)
new_counter = Counter(new_ds_STOPcodons)

# Create lists of frequencies
obs_values = [original_counter.get(codon, 0) for codon in stop_codons]
new_values = [new_counter.get(codon, 0) for codon in stop_codons]

# Perform chi-square test
chi2_stat, p_value = chi2_contingency([obs_values, new_values])[0:2]

print("Stop codon occurence in unodified transcripts:")
print(Counter(original_stop_codons))
print("Down-stream stop codon occurence after stoploss:")
print(Counter(new_ds_STOPcodons))
print(f"Chi-square statistic: {chi2_stat}")
print(f"p-value: {p_value}")


Plot stop codons

In [55]:
# Plot
# Get counts
original_counts = [Counter(original_stop_codons).get(codon, 0) for codon in stop_codons]
new_counts = [Counter(new_ds_STOPcodons).get(codon, 0) for codon in stop_codons]
x = np.arange(len(stop_codons))
width = 0.35
fig, ax = plt.subplots(figsize=(5, 6))
# Create bars
ax.bar(x - width/2, original_counts, width, label='CDS', color='#333333')
ax.bar(x + width/2, new_counts, width, label="3'UTR", color='#a4bad0')
# Customize the plot
ax.set_ylabel('Occurrences')
ax.set_title('Stop Codon Comparison', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(stop_codons)
ax.legend()
p_value_text = f'Chi-square p-value: {p_value:.4f}'
if p_value < 0.05:
    p_value_text += ' *'
ax.text(0.5, 0.95, p_value_text, transform=ax.transAxes, 
        horizontalalignment='center')
plt.tight_layout()
plt.savefig(wdir+"STOPcodon_occurence.png")
plt.close()

Stacked barplot of frequencies

In [56]:
import scipy.stats as stats
# Function to count the proportions of each stop codon in a list
def count_proportions(lst):
    stop_codons = ['TGA', 'TAA', 'TAG']
    count = {codon: lst.count(codon) for codon in stop_codons}
    total = sum(count.values())
    proportions = {codon: count[codon] / total for codon in stop_codons}
    return proportions

original_props = count_proportions(original_stop_codons)
ds_props = count_proportions(new_ds_STOPcodons)

# Create a DataFrame for easy plotting
df_prop = pd.DataFrame({
    'TGA': [original_props['TGA'], ds_props['TGA']],
    'TAA': [original_props['TAA'], ds_props['TAA']],
    'TAG': [original_props['TAG'], ds_props['TAG']],
}, index=['CDS', "3'UTR"])

observed = [list(original_props.values()), list(ds_props.values())]
# Perform the Chi-squared test
chi2_stat, p_value, dof, expected = stats.chi2_contingency(observed)

# Plot the stacked bar chart
ax = df_prop.plot(kind='bar', stacked=True, figsize=(5, 6), color=['#a3bac0', '#49636b', '#1d282b'])
ax.set_title('Stop Codon Proportion Comparison', pad=40)
plt.ylabel("Proportion")
plt.xticks(rotation=0)
plt.legend(title='Stop Codons')
plt.tight_layout()
plt.figtext(0.35, 0.9, f'Chi-squared p-value = {p_value:.4f}', fontsize=12, color='black')
plt.savefig(wdir+"STOPcodon_proportions.png")
plt.close()


In [57]:
original_props

{'TGA': 0.49913564596058546,
 'TAA': 0.2774576466520687,
 'TAG': 0.22340670738734586}

In [58]:
gene_symbol = "PTMA"
entry = trimmed_UTR_dict[gene_symbol]
print(f"Gene: {gene_symbol}")
print(f"Header: {entry['header']}")
print(f"CDS: {entry['CDS']}")
print(f"cDNA: {entry['cDNA']}")
print(f"3UTR: {entry['3UTR']}")
print(f"UTR: {entry['UTR_trimmed']}")

#### Put together the new peptide sequences with all possible mutations of the STOP codon
- Get last 8 codons from cds (including STOP in last position)
- Depending on which last STOP codon (TAA, TGA, TAG), mutated the last position
- Join mutated sequence with the trimmed 3'UTR tail
- translate the sequneces and store the new AA in fasta (us as headers gene name and STOP codon mutation)

In [59]:
def generate_mutated_sequences_with_utr(input_dict):
    """
    Apply utations to the STOP codon, and append the trimmed UTR sequence:
    Mutated sequence: base_seq_without_stop + mutated_STOP + utr_3_seq

    Returns a new dictionary with mutated sequences for each gene.
    """
    nonsilentSNV_dict ={ 
        "TAG" : ["GAG","TGG","AAG","CAG","TCG","TAC","TTG"],    # TAT codes as TTG for Tyr
        "TAA" : ["GAA","AAA","CAA","TCA","TAC","TTA"],          # TAU codes as TAC for Tyr 
        "TGA" : ["GGA","TGG","AGA","TCA","TGC","TTA"],          # CGA codes as AGA for Arg, TGT codes as TGC for Cys
    }
    # Set counters for the STOP codons
    n_TAG=0
    n_TAA=0
    n_TGA=0

    mutated_dict = {}

    for gene_symbol, info in input_dict.items():
        cds_seq = info.get('CDS')
        utr_3_seq = info.get('UTR_trimmed')

        if cds_seq and utr_3_seq:
            # Extract the last 9 codons (27 nucleotides), the last 3 should be STOP
            last_8_codon_seq = cds_seq[-27:] 
            stop_codon = last_8_codon_seq[-3:] 

            # Ensure the STOP codon is TAG, TAA or TGA
            if stop_codon in nonsilentSNV_dict:
                # counter for STOP codons
                if   stop_codon == "TAG": n_TAG += 1
                elif stop_codon == "TAA": n_TAA += 1
                elif stop_codon == "TGA": n_TGA += 1

                mutated_STOPcodons = nonsilentSNV_dict[stop_codon] # get the mutated codons
                base_seq_without_stop = last_8_codon_seq[:-3]  # Drop STOP codon 

                # Store cds and the mutated sequences for this gene
                not_mutated_seq = []
                mutated_seqs_full = []
                mutated_seqs_8 = []

                # Generate mutated sequences
                for mutated_codon in mutated_STOPcodons:
                    # Full mutated sequence: CDS + mutated_STOP + utr_3_seq
                    mutated_seq_full = cds_seq[:-3] + mutated_codon + utr_3_seq
                    # Last 8 canonical Aa + mutated_STOP + utr_3_seq
                    mutated_seq_8 = base_seq_without_stop + mutated_codon + utr_3_seq
                    # Make list with all mutated sequences for the gene
                    not_mutated_seq.append(cds_seq)
                    mutated_seqs_full.append(mutated_seq_full)
                    mutated_seqs_8.append(mutated_seq_8)

                mutated_dict[gene_symbol] = {
                    'header': info.get('header'),  
                    'cds_seq' : not_mutated_seq,
                    'mutated_seqs_full': mutated_seqs_full , 
                    'mutated_seqs_8': mutated_seqs_8  
                }
            else:
                print(f"Warning: Unrecognized STOP codon '{stop_codon}' for gene {gene_symbol}. Skipping mutations.")
        else:
            print(f"Warning: Missing CDS or 3' UTR sequence for gene {gene_symbol}.")

    print(f"STOP codon counts: TAG = {n_TAG}, TAA = {n_TAA}, TGA = {n_TGA}")

    return mutated_dict

print("\n\nCreate mutated sequences from CDS+mutated STOP+3'UTR to next STOP.")
mutated_seq_dict = generate_mutated_sequences_with_utr(trimmed_UTR_dict)


#### STEP 3.4: Translate the mutated seq to peptide format and store in fasta outfile
- Translate the mutated CDS sequences into peptides
- Make unique IDs for each new seq: [gene_symbol].X_[ENST_ID]


In [60]:
def update_header(header, gene_symbol, i):
    """
    Updates the header so that the first string after '>' is 'gene_symbol+i', and the rest of the header remains unchanged.
    """
    # Split the header by the first space to separate the ENST ID from the rest of the header
    parts = header.split(' ', 1)
    
    # Create the new header by replacing the ENST ID with 'gene_symbol+i'
    new_header = f">{gene_symbol}.{i} {parts[1]}"  # Reconstruct the header
    
    return new_header


In [61]:
# translate seq from cds nuc to AA pep
from Bio.Seq import Seq

def translate_mutated_cds_to_pep(nuc_dict):
    """
    Translates the mutated sequences from mutated_seq_dict to peptide sequences and stores them in a new dictionary.
    A new dictionary with gene_symbol as keys and the translated peptide sequence and header as values.
    """
    peptide_dict = {}

    genes_inframe_stp = []

    for gene_symbol, info in nuc_dict.items():
        cds_seq_list = info.get('cds_seq')  # Get the full mutated CDS nucleotide sequence
        nuc_seq_full_list = info.get('mutated_seqs_full')  # Get the full mutated CDS nucleotide sequence
        nuc_seq_8_list = info.get('mutated_seqs_8')  # Get the shortened CDS nucleotide sequence
        header = info.get('header')     # Get the header information

        if len(nuc_seq_full_list) == 0:
            print(f"No mutated sequences found. Skip {gene_symbol}")
            break
        
        for i in range(len(nuc_seq_full_list)):
            # Translate not mutated CDS
            cds_seq = cds_seq_list[i]
            cds_seq_obj = Seq(cds_seq) # using Biopython's Seq object724594
            if str(cds_seq[:3]).upper() != "ATG":
                print(gene_symbol)
                print(f"ERROR: No Start codon in not mutated cds seq:",cds_seq)
            else:
                # check if we have an extra in-frame STOP codon
                n = len(cds_seq)
                stp_counter = 0
                for j in range(0, n - n % 3, 3):
                    codon = cds_seq[j : j + 3]
                    if codon in stop_codons:
                        stp_counter += 1
                
                if stp_counter >= 2:
                    genes_inframe_stp.append(gene_symbol)
                else:
                    cds_peptide = str(cds_seq_obj.translate(to_stop=True))  # Translate from ATG until the first STOP codon

            # Translate full mutated CDS
            nuc_seq_full = nuc_seq_full_list[i]
            nuc_seq_full_obj = Seq(nuc_seq_full) # using Biopython's Seq object724594
            if str(nuc_seq_full[:3]).upper() != "ATG":
                print(gene_symbol)
                print(f"ERROR: No Start codon in seq:",nuc_seq_full)
            else:
                # check if we have an extra in-frame STOP codon
                n = len(nuc_seq_full)
                stp_counter = 0
                for j in range(0, n - n % 3, 3):
                    codon = nuc_seq_full[j : j + 3]
                    if codon in stop_codons:
                        stp_counter += 1
                
                if stp_counter >= 2:
                    genes_inframe_stp.append(gene_symbol)
                else:
                    full_peptide_seq = str(nuc_seq_full_obj.translate(to_stop=True))  # Translate from ATG until the first STOP codon

            # Translate shortened mutated CDS
            nuc_seq_8 = nuc_seq_8_list[i]
            nuc_seq_obj = Seq(nuc_seq_8)
            peptide_8_seq = str(nuc_seq_obj.translate(to_stop=True))  # Translate until the first STOP codon
            # comment: we cannot use 'cds=True' in the translate function, as we cut of the ATG together with 
            # the beginning of the cds and only kept its last 8 codons (incl the previous STOP codon) 
 
            # make unique ID for mutated seq
            updated_header = update_header(header, gene_symbol, i)
            
            
            # Store the peptide sequence and the header in the new dictionary
            peptide_dict[updated_header] = {
                # not mutated cds
                'pep_cds' : cds_peptide, 
                'cds_seq' : cds_seq,
                # cds + elongations
                'pep_AA_full': full_peptide_seq,
                'mutated_cds_full': nuc_seq_full,
                # last 8 AA of cds + elongations
                'pep_AA_8': peptide_8_seq,
                'mutated_cds_8': nuc_seq_8,
                # pep elongation only
                'elongation' : peptide_8_seq[9:]
            }

    # remove gene with in-frame stop in cds
    print(f"For",len(set(genes_inframe_stp)),"genes, the cds contains an in-frame stop and will be dropped.")
    
    return peptide_dict

print("\n\nTranslate mutated sequences to amino acid.")
pep_cds_dict = translate_mutated_cds_to_pep(mutated_seq_dict)
print(f"We have mutated the CDS from {len(mutated_seq_dict.keys())} genes.")
print(f"This resulted in {len(pep_cds_dict.keys())} peptide sequences from STOPloss mutations.")

In [62]:
dict(list(pep_cds_dict.items())[0: 1]) 

{'>PKD1L2.0 cds chromosome:GRCh38:16:81170289:81181329:-1 gene:ENSG00000166473.17 gene_biotype:polymorphic_pseudogene transcript_biotype:protein_coding gene_symbol:PKD1L2 description:polycystin 1 like 2 (gene/pseudogene) [Source:HGNC Symbol;Acc:HGNC:21715]': {'pep_cds': 'MGEDSPVAMFSWYLDNTPTEQAEPLLDACRLRGFWPRSLTLLQSNTSTLLLNSSFLQSRGEVIRIRATALTRHAYGEDTYVISTVPPREVPACTIAPEEGTVLTSFAIFCNASTALGPLEFCFCLESGSCLHCGPEPALPSVYLPLGEENNDFVLTVVISATNRAGDTQQTQAMAKVALGDTCVEDVAFQAAVSEKIPTALQGEGGPEQLLQLAKAVSSMLNQEHESQGSGQSLSIDVRQKVREHVLGSLSAVTTGLEDVQRVQELAEVLREVTCRSKELTPSAQGSCMGDSWEGAPPAAHVSHAR',
  'cds_seq': 'ATGGGGGAGGACTCTCCAGTGGCTATGTTCAGCTGGTATTTGGACAACACCCCAACAGAGCAGGCTGAGCCCCTCCTGGATGCCTGCAGACTCAGAGGATTTTGGCCAAGGTCCTTAACCCTCCTCCAGAGCAACACCTCCACGTTGCTGTTGAACAGCTCGTTTCTGCAGTCCCGGGGAGAGGTCATCCGAATCAGAGCCACAGCACTGACCAGGCATGCCTATGGGGAGGACACCTATGTGATCAGCACTGTGCCTCCCCGTGAGGTGCCTGCCTGCACTATTGCCCCAGAGGAGGGCACCGTTCTGACGAGCTTTGCCATCTTCTGCAACGCCTCCACAGCCCTGGGACCCCTGGAGTTCTGCTTCTGTCTGGAATCAGGTTCCTGCCTACACTGTGGCCCT

## Step 4: Write Fasta Files
Function *write_fasta_files()* creates 2 output files:
        - peptide seq in amino acids
        - mutated and elongated CDS in nucleotides

The final two fasta files contain the exact same sequneces of the mutated sequences
in amino acid (pep) and nucleotide (CDS) format.

**Mutated sequence: [8 last codons of CDS] + [mutated STOP codon] + [3'UTR to downstream STOP codon]**

In [63]:
def write_fasta_files(seq_dict,out_path):
    """
    Write two FASTA files: one for peptide sequences and one for mutated CDS sequences.
    """
    # the last 8 canonical AA + elongations
    cds_fasta_path = outpath+'CDS_elongation_seq.fasta'
    pep_fasta_path = outpath+'peptide_elongations.fasta'
    # the whole cds + elonagtions
    pep_fasta_full_path = outpath+'peptide_whole_transcript_elongations.fasta'
    cds_fasta_full_path = outpath+'CDS_whole_transcript_elongation_seq.fasta'
    # only the cds (not mutated)
    pep_fasta_notmutated_path = outpath+'CDS_peptide_notmutated.fasta'
    cds_fasta_notmutated_path = outpath+'CDS_transcript_notmutated.fasta'

    with open(pep_fasta_path, 'w') as pep_fasta, open(pep_fasta_full_path, 'w') as pep_full_fasta, open(pep_fasta_notmutated_path, 'w') as pep_fasta_notmutated, open(cds_fasta_path, 'w') as cds_fasta_8, open(cds_fasta_full_path, 'w') as cds_fasta_full,  open(cds_fasta_notmutated_path, 'w') as cds_fasta_notmutated:
        for header, sequences in seq_dict.items():
            pep_seq_full = sequences.get('pep_AA_full')
            pep_seq_8 = sequences.get('pep_AA_8')  # Get peptide sequence
            mutated_cds_seq_full = sequences.get('mutated_cds_full')  # Get mutated CDS sequence
            mutated_cds_seq_8 = sequences.get('mutated_cds_8')  # Get mutated CDS sequence

            # Write the peptide sequence to the peptide FASTA file
            if pep_seq_full:
                pep_full_fasta.write(f"{header}\n")
                pep_full_fasta.write(f"{pep_seq_full}\n")
            
            if pep_seq_8:
                pep_fasta.write(f"{header}\n")
                pep_fasta.write(f"{pep_seq_8}\n")

            # Write the mutated CDS sequence to the CDS FASTA file
            if mutated_cds_seq_full:
                cds_fasta_full.write(f"{header}\n")
                cds_fasta_full.write(f"{mutated_cds_seq_full}\n")

            if mutated_cds_seq_8:
                cds_fasta_8.write(f"{header}\n")
                cds_fasta_8.write(f"{mutated_cds_seq_8}\n")

            # Write the CDS sequence (without mutations) to the CDS FASTA file
            cds_seq = sequences.get('cds_seq') 
            if cds_seq:
                cds_fasta_notmutated.write(f"{header}\n")
                cds_fasta_notmutated.write(f"{cds_seq}\n")

            cds_pep = sequences.get('pep_cds') 
            if cds_pep:
                pep_fasta_notmutated.write(f"{header}\n")
                pep_fasta_notmutated.write(f"{cds_pep}\n")

print("\n\nSave amino acids and mutated nucleotide sequences to fasta files.")
write_fasta_files(pep_cds_dict, outpath )

In [64]:
# file with Gene Symbol and only elongation (without mutated STOP)
def consolidate_gene_elongation(input_dict):
    # Create a new dictionary to store the consolidated results
    consolidated_dict = {}
    
    # Iterate through the input dictionary
    for full_key, value_dict in input_dict.items():
        # Extract the base gene name (removing version number)
        base_gene = full_key.split(' ')[0].rsplit('.', 1)[0]
        
        # If this base gene is not in the consolidated dict, add it
        if base_gene not in consolidated_dict:
            # Check if 'elongation' exists in the current value dictionary
            if 'elongation' in value_dict:
                consolidated_dict[base_gene] = value_dict['elongation']
    
    # print to fasta file
    pep_elongation_fasta_path = outpath+'Elongation_only_pep.fasta'
    with open(pep_elongation_fasta_path, 'w') as pep_elongation_fasta:
        for header, sequence in consolidated_dict.items():
            pep_elongation_fasta.write(f"{header}\n")
            pep_elongation_fasta.write(f"{sequence}\n")

    return consolidated_dict

gene_elong_dict = consolidate_gene_elongation(pep_cds_dict)
len(gene_elong_dict) # 16073 genes
dict(list(gene_elong_dict.items())[0:10]) 

{'>PKD1L2': 'EGLFAQTLTPASAGEQVEGVTESH',
 '>INTS3': 'GPAFPIPPPAGLPSPSW',
 '>KIR2DL3': 'SKVVSCP',
 '>KIR2DL4': 'RRESPS',
 '>KIR2DL1': 'APQSGLEGVF',
 '>KIR3DL1': 'APQSGLEDIF',
 '>KIR2DS3': 'LDHCVFTQRKITPPSQRPKTPPTDSSMYIELPNAESRSKAVFCPRAPQSGLEGIF',
 '>KIR2DL2': 'APQSGLEGVF',
 '>PCMTD2': 'VSCLKGGNRKSRLLSVKFVLPVCC',
 '>KIR3DP1': 'FCSICLK'}

In [65]:
print("\n\nVersion:")
print(sys.version)

sys.stdout = old_stdout

log_file.close()

print("All done. Gute Nacht.")